In [61]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # BE QUIET!!!! (info and warnings are not printed)
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np 
import matplotlib.pyplot as plt

import math
import sys
import numpy.linalg as la
import numpy.matlib
import tensorflow.compat.v1 as tf

rng = np.random.RandomState(seed=None)
import pickle   

In [62]:
# out_list = []

Parameter Initialization

In [63]:
system_params   = {'P': 10.0,   # Average codeword symbol power constraint
                 'R': 0.3,      # Rate
                 'L': 256,      # Number of sections
                 'M': 512,       # Columns per section
                 'snr_rx': 31,  # SNR at the receiver  (not in dB)   (SNR of 15 gives a capacity of 2 bits/sec)
                 'dist': 1      # 0 for flat and 1 for exponential
                }

# AMP Encoding

In [64]:
def power_dist(power_dist_params):
    power_dist_params_list = ['P','L','n','capacity','dist','R']
    P,L,n,capacity,dist, R =  map(power_dist_params.get, power_dist_params_list) 

    #Flat power Distribution
    P_vec_flat = P/L * np.ones((L,1))  # Considering Flat power distribution. Can also do exponential distribution

    # Exponential Power distribution
    P_vec_unnorm = np.zeros(L)
    #kappa = 2*capacity
    for i in range(L):
        P_vec_unnorm[i] = np.power(2, (-2*R*i)/L )
    pow_exp = np.sum(P_vec_unnorm)
    P_vec_exp = (P/pow_exp) * P_vec_unnorm    # Normalising such that the sum of Pi's = P

    if dist==0:
        beta_non_zero_values = np.sqrt(n*P_vec_flat)
        P_vec = P_vec_flat
    else:
        beta_non_zero_values = np.sqrt(n*P_vec_exp).reshape((L,1))      #use for exponential power distribution
        P_vec = P_vec_exp.reshape((L,1))

    return beta_non_zero_values,P_vec

In [65]:
def generate_message(system_params,rng,cols):
    system_params_list = ['P','R','L','M','snr_rx','dist']
    P,R,L,M,snr_rx,dist = map(system_params.get, system_params_list) 
    N = L*M

    # Sparse Message vectors with ones (number of msg vectors=cols)
    beta = np.zeros((L*M,cols))
    beta_val = np.zeros((L*M, cols))

    # Same Sparse Message vectors with power allocated non-zero values
    beta2 = np.zeros((L*M,cols))
    beta2_val = np.zeros((L*M, cols))

    #Section sizes 
    bit_len = int(round(L*np.log2(M)))
    logM = int(round(np.log2(M)))
    sec_size = logM
    L = bit_len // sec_size

    # Actual rate
    n = int(round(bit_len/R))
    R_actual = bit_len / n      

    # SNR and noise variance calculation
    #snr_rx = np.power(10., snr_rx/10.) * (N/n)              # *log2M (not sure if it should be there)  # according to LISTA code
    #snr_rx = P/awgn_var

    capacity = 0.5 * np.log2(1 + snr_rx)
    
    awgn_var = P/snr_rx
    sigma = np.sqrt(awgn_var)
    
    SNR_params = {'snr_rx':snr_rx,
                 'awgn_var':awgn_var,
                 'capacity':capacity}

    # Power allocation
    power_dist_params = {'P':P,
                        'L':L,
                        'n':n,
                        'capacity':capacity,
                        'dist':dist,        # dist=0 for flat and 1 for exponential
                        'R': R_actual
                         }
    beta_non_zero_values, P_vec = power_dist(power_dist_params)

    for i in range(2*cols):
        bits_in = rng.randint(2, size=bit_len)
        beta0 = np.zeros(L*M)   #placeholder for beta and beta_val (nonzeros =1)
        beta1 = np.zeros(L*M)   #placeholder for beta2 and beta2_val (nonzeros = power allocated values)

        for l in range(L):
            bits_sec = bits_in[l*sec_size : l*sec_size + logM]
            assert 0<logM<64
            idx = bits_sec.dot(1 << np.arange(logM)[::-1])
            beta0[l*M + idx] = 1
            beta1[l*M + idx] = beta_non_zero_values[l]

        # storing first 'cols' vectors in beta(beta2) and the next 'cols' vectors in beta_val(beta2_val)
        if i < cols:
            beta[:,i-1] = beta0
            beta2[:,i-1] = beta1
        else:
            beta_val[:,i - cols] = beta0
            beta2_val[:, i - cols] = beta1

        delim = np.zeros([2,L])
        delim[0,0] = 0
        delim[1,0] = M-1

        for i in range(1,L):
            delim[0,i] = delim[1,i-1]+1
            delim[1,i] = delim[1,i-1]+M

    new_system_params = {'n':n,
                        'R_actual': R_actual,
                        'delim': delim,
                        'P_vec':P_vec
                        }
                        
    return beta, beta2, beta_val, beta2_val, new_system_params,SNR_params, 


Extracting all the system parameters 

In [66]:
system_params_list = ['P','R','L','M','snr_rx']
P,R,L,M,snr_rx = map(system_params.get, system_params_list)
N = L*M 

# Generating the message vectors
cols = 100
beta, beta2, beta_val, beta2_val, new_system_params, SNR_params = generate_message(system_params,rng,cols)

# # vectors with 1s
# beta = beta.reshape(N,)
# beta_val = beta_val.reshape(N,)

# # vectors with power allocated values (need to be used for matrix received vector generation)
# beta2 = beta.reshape(N,)
# beta2_val = beta_val.reshape(N,)

# Extracting the new system parameters
new_system_params_list = ['n','R_actual','delim','P_vec']
n, R, delim,P_vec = map(new_system_params.get, new_system_params_list)

# Extracting the SNR params
SNR_params_list = ['snr_rx','awgn_var','capacity']
snr_rx,awgn_var,capacity = map(SNR_params.get, SNR_params_list)
sigma = np.sqrt(awgn_var)

Generating the A matrix and the received vector (after adding noise)

In [67]:
A = np.random.normal(size=(n, N), scale=1.0 / math.sqrt(n)).astype(np.float32)
#A = A_unn/np.linalg.norm(A_unn,axis=0) 
y = np.matmul(A,beta2) + np.random.normal( size = (n,cols), scale = math.sqrt(awgn_var) )

# need to check how to get ||Ax||^2 = ||x||^2

# AMP Decoder

In [68]:
def eta(beta_hat, P_vec, tau, delim):
    beta_th = np.zeros(np.shape(beta_hat))
    rows,cols = beta_hat.shape 
    L = np.size(P_vec)
    M = rows/L

    for i in range(cols):
        s1 = beta_hat[:,i]
        for j in range(L):
            beta_section = s1[ int(delim[0,j]):int(delim[1,j]+1)]
            beta_th_section = np.zeros(int(M))
            denom = np.sum( np.exp( (beta_section* np.sqrt(n*P_vec[j]))/(tau**2) ) )
            for k in range(int(M)):
                num = np.exp( (beta_section[k]* np.sqrt(n*P_vec[j]))/(tau**2) )
                beta_th_section[k] = num/denom
            beta_th[int(delim[0,j]):int(delim[1,j]+1), i] = beta_th_section

    return beta_th        

In [69]:
def tau_calculate(sigma_sq,delim, P, P_vec, T_star, n):
    tau_vec = np.zeros([T_star,1])
    x = np.zeros([T_star,1])
    tau_vec[0] = sigma_sq + P
    ITR = 10^4

    for t in range(1,T_star):
        exp_vec = np.zeros([np.size(P_vec),1])

        for i in range(ITR):
            U = rng.randn(int(delim[-1,-1]))
            for l in range(np.size(P_vec)):
                # Numerator
                num_1 =  np.sqrt(n*P_vec[l],dtype=np.float128)/tau_vec[t-1]
                num_21 = U[int(delim[0,l])]
                num = np.exp( num_1 * (num_1 + num_21 ),dtype=np.float128)

                # Denominator
                denom_21 = U[ int(delim[0,l]+1) : int(delim[1,l]+1) ]
                denom_2 = np.sum( np.exp( np.multiply(num_1,denom_21) ), dtype=np.float128)
                denom = num + denom_2

                exp_vec[l] = exp_vec[l] + np.divide(num,denom, dtype=np.float128)

        exp_vec = np.divide(exp_vec,ITR)
        x1 = np.ndarray.item(np.float128(np.matmul(P_vec.T,exp_vec)/P)) # getting saved as an (1,1) ndarray
        
        tau_vec[t] = sigma_sq + P*(1-x1)
        x[t] = x1

    return tau_vec, x

In [70]:
# test_arr = beta[:,0]
# nonzero_beta = np.nonzero(test_arr)

In [71]:
T_star = int(np.ceil((2*capacity)/np.log2(capacity/R)) + 1)
tau_vec1, x= tau_calculate(awgn_var, delim,P,P_vec,T_star,n)

In [72]:
tau_vec = np.concatenate((tau_vec1[0].reshape([1,1]),tau_vec1), axis=0)

In [73]:
beta_T = np.zeros((N,cols))
beta_hat = np.zeros((N,cols))
z = np.zeros((n,cols))


# beta_hat_nz_cdf = np.zeros( [int(np.size(nonzero_beta)),T_star] )

for t in range(T_star):
    z = y - np.matmul(A, beta_hat) + (z/tau_vec[t])*(P - (np.linalg.norm(beta_hat)**2)/n )
    beta_hat = beta_hat + np.matmul(A.T,z)
    beta_hat = eta(beta_hat,P_vec, tau_vec[t],delim)
    
    # beta_hat_test = beta_hat[nonzero_beta,0].reshape([-1,1])
    # beta_hat_nz_cdf[:,t] = np.divide(beta_hat_test,np.sqrt(n*P_vec)).reshape([L,])
    
    
for j in range(L):
    loc = np.argmax( beta_hat[ int(delim[0,j]) : int(delim[1,j]) ], axis=0 )
    for k in range(np.size(loc)):
        beta_T[ int(delim[0,j]) + loc[k],k ] = 1
        

In [74]:
check = beta-beta_T
check_sum = np.sum(np.absolute(check),axis=0)/2
check_avg = np.average(check_sum)
print(check_avg)

5.29


Saving the Outputs

In [75]:
# output_params = system_params.copy()
# output_params['check_avg']=check_avg

# filename = 'Avg_ction_error_rate_output'

# infile = open(filename,'rb')
# out_list = pickle.load(infile)

# out_list.append(system_params)

# # outfile = open(filename,'wb')
# with open(filename,'wb') as outfile:
#     pickle.dump(out_list,outfile)